In [1]:
import pandas as pd
import numpy as np
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.middleware.cors import CORSMiddleware

from signlens.preprocessing.preprocess import group_pad_sequences, load_and_pad, decode_labels
from signlens.preprocessing.data import load_relevant_data_subset, load_data_subset_csv
from utils.model_utils import load_model

2024-03-22 18:17:55.730541: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
pq_path = '/Users/max/code/benoitfrisque/signlens/raw_data/asl-signs/train_landmark_files_noface/22343/1130422569.parquet'

In [3]:
pd.read_parquet(pq_path)

,frame,row_id,type,landmark_index,x,y,z
468,61,61-left_hand-0,left_hand,0,NaN,NaN,NaN
469,61,61-left_hand-1,left_hand,1,NaN,NaN,NaN
470,61,61-left_hand-2,left_hand,2,NaN,NaN,NaN
471,61,61-left_hand-3,left_hand,3,NaN,NaN,NaN
472,61,61-left_hand-4,left_hand,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...
3253,66,66-right_hand-16,right_hand,16,NaN,NaN,NaN
3254,66,66-right_hand-17,right_hand,17,NaN,NaN,NaN
3255,66,66-right_hand-18,right_hand,18,NaN,NaN,NaN
3256,66,66-right_hand-19,right_hand,19,NaN,NaN,NaN


In [4]:
processed_data = preprocess_and_pad_sequences_from_pq_list(pd.Series([pq_path]))
processed_data

<tf.Tensor: shape=(1, 100, 225), dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)>

In [7]:
model_name = "model 20240322-173411"
model = load_model(model_name)

model = model[0]

prediction = model.predict([processed_data])


Load latest model from local registry...

Load latest model from disk...
✅ Model loaded from local disk /Users/max/code/benoitfrisque/signlens/training_outputs/model 20240322-173411
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [6]:
processed_data

<tf.Tensor: shape=(1, 100, 225), dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)>

In [8]:
prediction

array([[0.12294942, 0.05086957, 0.09900542, 0.14869675, 0.08723761,
        0.05622363, 0.1207337 , 0.11545481, 0.08714273, 0.11168633]],
      dtype=float32)

In [9]:
word = decode_labels(prediction)
word

(['chair'], array([0.14869675], dtype=float32))